In [1]:
import pandas as pd
from src.plots import multi_line_graph, violin_plots, stacked_plot

In [2]:
df_1 = pd.read_csv("data/output/Corentin/accelerometer4_data.csv")
df_2 = pd.read_csv("data/output/Matthieu/accelerometer3_data.csv")
df_3 = pd.read_csv("data/output/Leo/accelerometer8_data.csv")

In [3]:
multi_line_graph([df_1, df_2, df_3],["Corentin", "Matthieu", "Leo"])

In [4]:
violin_plots([df_1, df_2, df_3],["Corentin", "Matthieu", "Leo"])

In [5]:
multi_line_graph([df_2, df_3],["Matthieu", "Leo"])

In [6]:
stacked_plot([df_1, df_2, df_3],["Corentin", "Matthieu", "Leo"])